# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [31]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [32]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event_data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# Ignoring hidden files like .ipynb_checkpoints present in the /event_data folder

for root, dirs, files in os.walk(filepath):
    if ('.ipynb_checkpoints' in root):
        continue    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)


/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [33]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


#### check the number of rows in your csv file

In [34]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [35]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [36]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [37]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Data Model:

### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and     itemInSession = 4

#### Code Query1 : select artist_name,song ,song_length from songplay_info_session where sessionid=338 and iteminsession=4;
#### Table Name : songplay_info_session
* col1: sessionid 
* col2: iteminsessionong
* col3: artist_name
* col4: song
* col5: song_length
* PRIMARY KEY : sessionid,iteminsession
#### Based on the query, sessionid and iteminsession combinely identify the record unique. One sessionid has many iteminsessions so both sessionid and iteminsession define unique records in table

In [38]:
query = "CREATE TABLE IF NOT EXISTS songplay_info_session "
query = query + "(sessionid int,\
                  iteminsession int,\
                  artist_name text,\
                  song text, \
                  song_length float,\
                  PRIMARY KEY (sessionid, iteminsession))"      
             
try:
    session.execute(query)
except Exception as e:
    print(e)                    

### Data  ingestion into songplay_info_session table from event_datafile_new.csv file

In [39]:
# Insert data into songplay_info_session table from event_datafile_new.csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_info_session(\
                                                sessionid,\
                                                iteminsession,\
                                                artist_name,\
                                                song,\
                                                song_length)"
                                
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9], float(line[5])))

#### Add in the SELECT statement to verify the data was entered into the table

In [41]:
query = "SELECT artist_name,song ,song_length \
         FROM songplay_info_session \
         WHERE \
         sessionid=338 and iteminsession=4"
try:
    rows = session.execute(query)
    result = pd.DataFrame(rows._current_rows,columns=['artist_name','song','song_length'])
    print(result)
except Exception as e:
    print(e)


  artist_name                             song  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## Data Model :

### Code Query 2 : select artist,song,firstname,lastname from user_playlist_session where userid = 10 and sessionid = 182

#### Table name : user_playlist_session
* col1 : user_id
* col2: sessionid
* col3: iteminsession
* col4: artist_name
* col5: song
* col6: firstname
* col7: lastname
* PRIMARY KEY : ((userid,sessionid),iteminsession)
#### Based on the query statement,userid and sessionid combinely defines the unique record in the table and as per data retrival        on sorted iteminsession, the clustering column would be iteminsession

#### Create user_playlist_session Table :

In [42]:
query = "CREATE TABLE IF NOT EXISTS user_playlist_session "
query = query + "(userid int,\
                  sessionid int,\
                  iteminsession int,\
                  artist_name text,\
                  song text,\
                  firstname text,\
                  lastname text,\
                  PRIMARY KEY ((userid,sessionid),iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert data into user_playlist_session Table from event_datafile_new.csv file

In [43]:
## Insert data into music table from event_datafile_new.csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_playlist_session(\
                                                userid,\
                                                sessionid,\
                                                iteminsession,\
                                                artist_name,\
                                                song,\
                                                firstname,\
                                                lastname)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0], line[9], line[1], line[4]))

#### Do SELECT on table to validate the data have been inserted into the table

In [48]:
query = "SELECT artist_name,song,firstname,lastname \
         FROM user_playlist_session \
         WHERE \
         userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
    result = pd.DataFrame(rows._current_rows,columns=['artist_name','song','firstname','lastname'])
    print(result)
except Exception as e:
    print(e)


         artist_name                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


##  Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Data Model:

#### Code Query 3: select firstname,lastname from songplay_users_info where song='All Hands Against His Own'
#### Table name: songplay_users_info
* col1: song
* col2: userid
* col3: firstname
* col4: lastname
* col5: gender
* PRIMARY KEY : (song,userid)
#### In this table,song and userid combinely define unique record in the table, as alone song can not identify uniqueness per record.

#### CREATE songplay_users_info table:

In [45]:
query = "CREATE TABLE IF NOT EXISTS songplay_users_info "
query = query + "(song text,\
                  userid int,\
                  firstname text,\
                  lastname text,\
                  gender text, \
                  PRIMARY KEY (song,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)   
                   

#### Insert data into songplay_users_info table from event_datafile_new.csv file

In [46]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_users_info(\
                                            song,\
                                            userid,\
                                            firstname,\
                                            lastname,\
                                            gender)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, ( line[9], int(line[10]), line[1],line[4], line[2]))

#### Do a SELECT statement to verify the data was entered into the table

In [47]:
query = "SELECT firstname,lastname \
         FROM songplay_users_info \
         WHERE \
         song='All Hands Against His Own'"
try:
    rows = session.execute(query)
    result = pd.DataFrame(rows._current_rows,columns=['Firstname','Lastname'])
    print(result)
except Exception as e:
    print(e)
  

    Firstname Lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [49]:
## Drop the table before closing out the sessions
query_list = ["DROP TABLE songplay_info_session","DROP TABLE user_playlist_session ","DROP TABLE songplay_users_info"]
for query in query_list:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()